In [4]:
import pandas as pd
from sqlalchemy import create_engine

In [5]:
airline_codes_file = "Airline Codes.csv"
airline_codes_df = pd.read_csv(airline_codes_file)
airline_codes_df.head()

,Code,Description
0,19031,Mackey International Inc.
1,19032,Munz Northern Airlines Inc.
2,19033,Cochise Airlines Inc.
3,19034,Golden Gate Airlines Inc.
4,19035,Aeromech Inc.


In [6]:
flight_data_file = "Jan_2020_ontime 2.csv"
flight_data_df = pd.read_csv(flight_data_file)
flight_data_df.head()

,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN,...,DEST,DEP_TIME,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE,Unnamed: 21
0,1,3,EV,20366,EV,N48901,4397,13930,1393007,ORD,...,GRB,1003.0,0.0,1000-1059,1117.0,0.0,0.0,0.0,174.0,NaN
1,1,3,EV,20366,EV,N16976,4401,15370,1537002,TUL,...,ORD,1027.0,0.0,1000-1059,1216.0,0.0,0.0,0.0,585.0,NaN
2,1,3,EV,20366,EV,N12167,4404,11618,1161802,EWR,...,TYS,1848.0,0.0,1800-1859,2120.0,0.0,0.0,0.0,631.0,NaN
3,1,3,EV,20366,EV,N14902,4405,10781,1078105,BTR,...,IAH,1846.0,0.0,1800-1859,2004.0,0.0,0.0,0.0,253.0,NaN
4,1,3,EV,20366,EV,N606UX,4407,14524,1452401,RIC,...,IAH,1038.0,0.0,1000-1059,1330.0,0.0,0.0,0.0,1157.0,NaN


In [7]:
# Create a filtered dataframe from specific columns
premise_cols = ["OP_CARRIER_AIRLINE_ID", "TAIL_NUM", "OP_CARRIER_FL_NUM", "ORIGIN", "DEST", "DEP_TIME", "DEP_DEL15", "DAY_OF_MONTH", "DAY_OF_WEEK"]
premise_transformed= flight_data_df[premise_cols].copy()

# Rename the column headers
premise_transformed = premise_transformed.rename(columns={"OP_CARRIER_AIRLINE_ID": "Airline Id",
                                                          "TAIL_NUM": "Tail Number",
                                                          "OP_CARRIER_FL_NUM": "Flight Number",
                                                          "ORIGIN": "Departure Airport",
                                                          "DEST": "Destination Airport",
                                                          "DEP_TIME": "Departure Time",
                                                          "DEP_DEL15": "Departure Time",
                                                          "DAY_OF_MONTH": "Day of the Month",
                                                          "DAY_OF_WEEK":"Day of Week"
                                                           })

# Clean the data by dropping duplicates and setting the index
premise_transformed.drop_duplicates("id", inplace=True)
premise_transformed.set_index("id", inplace=True)

premise_transformed.head()

KeyError: Index(['id'], dtype='object')